# The first ML algorithms emerging

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

## Perceptron

The perceptron is a binary linear classifier that maps an input vector $ \mathbf{x} \in \mathbb{R}^n $ to an output $ y \in \{0, 1\} $ using a weight vector $ \mathbf{w} \in \mathbb{R}^n $ and bias $ b \in \mathbb{R} $. The decision function is:

$$
y = 
\begin{cases}
1 & \text{if } \mathbf{w}^\top \mathbf{x} + b \geq 0 \\
0 & \text{otherwise}
\end{cases}
$$

Learning updates the weights via:

$$
\mathbf{w} \leftarrow \mathbf{w} + \eta(y - \hat{y})\mathbf{x}
$$

where $ \eta $ is the learning rate and $ \hat{y} $ is the predicted label.


![Perceptron](images/perceptron_diagram.png)

In [2]:
class Perceptron:
    def __init__(self, n_features, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = np.zeros(n_features)
        self.bias = 0

    def predict(self, X):
        linear = np.dot(X, self.weights) + self.bias
        return np.where(linear >= 0, 1, 0)

    def fit(self, X, y):
        for _ in range(self.n_iters):
            for xi, target in zip(X, y):
                update = self.lr * (target - self.predict(xi))
                self.weights += update * xi
                self.bias += update

## Adaline

The Adaline (Adaptive Linear Neuron) is a binary classifier that uses a linear activation function. Given input vector $ \mathbf{x} \in \mathbb{R}^n $, weights $ \mathbf{w} \in \mathbb{R}^n $, and bias $ b \in \mathbb{R} $, the net input is:

$$
z = \mathbf{w}^\top \mathbf{x} + b
$$

The prediction is:

$$
\hat{y} = 
\begin{cases}
1 & \text{if } z \geq 0 \\
0 & \text{otherwise}
\end{cases}
$$

The weights are updated using the rule:

$$
\mathbf{w} \leftarrow \mathbf{w} + \eta \sum_i (y_i - \hat{y}_i) \mathbf{x}_i
$$

$$
b \leftarrow b + \eta \sum_i (y_i - \hat{y}_i)
$$

where $ \eta $ is the learning rate.


In [3]:
class Adaline:
    def __init__(self, n_features, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = np.zeros(n_features)
        self.bias = 0

    def net_input(self, X):
        return np.dot(X, self.weights) + self.bias

    def predict(self, X):
        return np.where(self.net_input(X) >= 0, 1, 0)

    def fit(self, X, y):
        for _ in range(self.n_iters):
            output = self.net_input(X)
            errors = y - output
            self.weights += self.lr * np.dot(X.T, errors)
            self.bias += self.lr * errors.sum()

## Perceptron vs Adaline

### Similarities:
- Both are classifiers for binary classification
- Both have linear decision boundary
- Both use a threshold function

### Differences:

- **Perceptron** uses a step function $ \phi(z) $ as its activation. It compares predicted class labels to the true labels and updates weights immediately after each misclassification (online learning). This can result in frequent updates within an epoch.

- **Adaline** uses a linear activation function $ \phi(z) = z$), which is simply the net input. It compares the continuous output to the true labels and performs batch updates at the end of each epoch, based on the total error across all training samples.

## Linear Regression

**Linear Regression** models the relationship between an input vector $ \mathbf{x} \in \mathbb{R}^n $ and a continuous target variable $ y \in \mathbb{R} $. It assumes the output is a linear combination of the inputs:

$$
\hat{y} = \mathbf{w}^\top \mathbf{x} + b
$$

The model is trained by minimizing the **mean squared error** (MSE):

$$
\mathcal{L} = \frac{1}{m} \sum_{i=1}^{m} \left( y_i - \hat{y}_i \right)^2
$$

where $ m $ is the number of training samples. The weights $ \mathbf{w} $ and bias $ b $ are updated using gradient descent to minimize the loss.


In [1]:
class LinearRegression:
    def __init__(self, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            y_pred = np.dot(X, self.weights) + self.bias
            dw = -(2/n_samples) * np.dot(X.T, (y - y_pred))
            db = -(2/n_samples) * np.sum(y - y_pred)
            self.weights -= self.lr * dw
            self.bias -= self.lr * db

    def predict(self, X):
        return np.dot(X, self.weights) + self.bias

## Support Vector Machines

Support Vector Machines (SVMs) are supervised learning models used for classification and regression. In binary classification, an SVM aims to find the optimal hyperplane that maximally separates two classes in the feature space.

Given training data $ \{(\mathbf{x}_i, y_i)\}_{i=1}^m $, where $ \mathbf{x}_i \in \mathbb{R}^n $ and $ y_i \in \{-1, +1\} $, the SVM solves the optimization problem:

$$
\min_{\mathbf{w}, b} \ \frac{1}{2} \|\mathbf{w}\|^2
$$

subject to the constraints:

$$
y_i(\mathbf{w}^\top \mathbf{x}_i + b) \geq 1 \quad \text{for all } i.
$$

This is known as the hard-margin SVM, suitable for linearly separable data.

For non-separable data, a soft-margin SVM introduces slack variables \( \xi_i \geq 0 \) and a penalty parameter \( C > 0 \):

$$
\min_{\mathbf{w}, b, \boldsymbol{\xi}} \ \frac{1}{2} \|\mathbf{w}\|^2 + C \sum_{i=1}^m \xi_i
$$

subject to:

$$
y_i(\mathbf{w}^\top \mathbf{x}_i + b) \geq 1 - \xi_i \quad \text{and} \quad \xi_i \geq 0.
$$

The decision function for prediction is:

$$
\hat{y} = \text{sign}(\mathbf{w}^\top \mathbf{x} + b)
$$

SVMs can be extended to non-linear classification using the kernel trick, which maps input data into a higher-dimensional space:

$$
K(\mathbf{x}_i, \mathbf{x}_j) = \phi(\mathbf{x}_i)^\top \phi(\mathbf{x}_j)
$$

Common kernels include the polynomial kernel and the radial basis function (RBF) kernel. The key idea is that the SVM algorithm depends only on inner products, so we don't need to compute $ \phi(\mathbf{x}) $ explicitly.

SVMs are powerful because they maximize the margin between classes and are robust to overfitting, especially in high-dimensional spaces.


In [2]:
class SVM:
    def __init__(self, lr=0.001, lambda_param=0.01, n_iters=1000):
        self.lr = lr
        self.lambda_param = lambda_param
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y):
        n_samples, n_features = X.shape
        y_ = np.where(y <= 0, -1, 1)
        self.weights = np.zeros(n_features)
        self.bias = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.weights) + self.bias) >= 1
                if condition:
                    self.weights -= self.lr * (2 * self.lambda_param * self.weights)
                else:
                    self.weights -= self.lr * (2 * self.lambda_param * self.weights - np.dot(x_i, y_[idx]))
                    self.bias -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.weights) + self.bias
        return np.sign(approx)

## K-Nearest Neighbors

K-Nearest Neighbors (KNN) is a non-parametric, instance-based learning algorithm used for classification and regression. Given a query point $ \mathbf{x} \in \mathbb{R}^n $, KNN identifies the $k$ closest training samples in the feature space using a distance metric, commonly the Euclidean distance:

$$
d(\mathbf{x}, \mathbf{x}_i) = \| \mathbf{x} - \mathbf{x}_i \|_2
$$

For classification, the algorithm assigns the most frequent label among the $k$ nearest neighbors:

$$
\hat{y} = \text{mode} \left( \{ y_i \ | \ \mathbf{x}_i \in \mathcal{N}_k(\mathbf{x}) \} \right)
$$

For regression, it typically takes the average of the neighbors’ target values:

$$
\hat{y} = \frac{1}{k} \sum_{\mathbf{x}_i \in \mathcal{N}_k(\mathbf{x})} y_i
$$

KNN is simple, interpretable, and effective for low-dimensional data, but becomes computationally expensive and less accurate in high-dimensional spaces.


In [1]:
from collections import Counter

class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        return np.array([self._predict(x) for x in X])

    def _predict(self, x):
        distances = [np.linalg.norm(x - x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_labels = [self.y_train[i] for i in k_indices]
        most_common = Counter(k_labels).most_common(1)
        return most_common[0][0]